In [ ]:
import random, time
import numpy as np
import pandas as pd
import multiprocessing as mp
import numba as nb

# Topics

0. Python Functions
1. Multiprocessing
2. NumPy
3. Pandas
4. Cython
5. Numba

## Python Functions


```
def function_name(optional_parameters):
    function_code
    return optional_return_variables
```

Functions encapsulate a set of instructions that can be reused. The following tips for faster Python all benefit from writing code that is well functionalized. Further, using functions are generally a good coding practice as they allow for the blocks of code to be written once and then reused many times. Then if the code needs to be updated it only needs to be updated once and not many times.

In [ ]:
def monte_carlo_pi(points):
    s = 0
    for _ in range(points):
        x = random.random()**2
        y = random.random()**2
        if x + y < 1:
            s += 1
    return 4. * float(s) / float(points)

In [ ]:
print(monte_carlo_pi(10000000))